In [3]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import json
import re

In [4]:
UserAgent().random

'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.137 Safari/4E423F'

In [11]:
# url = 'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&maxarea=20&minarea=5&offer_type=offices&office_type%5B0%5D=2&office_type%5B1%5D=5&p=2&region=1'

# url = 'https://www.cian.ru/'

url = 'https://www.cian.ru/snyat-ofis/'
# url = 'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&maxarea=20&minarea=5&offer_type=offices&office_type%5B0%5D=2&office_type%5B1%5D=5&p=2&region=1'


# html = requests.get(url, headers={'User-Agent':str(UserAgent().random)}).text

html = requests.get(url).text

soup = BeautifulSoup(html, 'lxml')

In [12]:
soup

<!DOCTYPE html>
<html lang="ru"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><style>/*! normalize.css v8.0.1 | MIT License | github.com/necolas/normalize.css */html{-webkit-text-size-adjust:100%;height:100%;font-family:Lato,Arial,sans-serif;font-size:14px;line-height:20px;color:#121212;-webkit-font-smoothing:antialiased;-moz-osx-font-smoothing:grayscale}body{margin:0;min-height:100%;display:flex;flex-direction:column}h1{font-size:2em;margin:.67em 0}hr{box-sizing:content-box;height:0;overflow:visible}code,kbd,pre,samp{font-family:monospace,monospace;font-size:1em}abbr[title]{border-bottom:none;text-decoration:underline dotted}b,strong{font-weight:bolder}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}button,input,optgroup,select,textarea{font-size:100%;line-height:1.15;margin:0}button,input{overflow:visible}button,select{text-transform:none}[type=butto

In [13]:
soup.find('div', {'data-name': 'Pagination'})

In [14]:
soup.find('div', {'class': '_32bbee5fda--wrapper--bKcEk'})

In [7]:
current_page = int(soup.find('div', {'data-name': 'Pagination'})\
                        .find('li', {'class': '_32bbee5fda--list-item--FFjMz _32bbee5fda--list-item--active--WifA5'}).text)

AttributeError: 'NoneType' object has no attribute 'find'

In [24]:
current_page

1

In [25]:
t = soup.findAll('script', {'type': 'text/javascript'})[0].decode_contents().strip().split("window._cianConfig['frontend-serp'] = ")[1]

IndexError: list index out of range

In [56]:
t = soup.findAll('script', {'type': 'text/javascript'})[3]\
    .decode_contents().strip().split("window._cianConfig['legacy-commercial-serp-frontend'] = ")[1]
t = t[1:-2]

In [61]:
ads = json.loads(t[[m.start() for m in re.finditer('"offers"', t)][0] + 9: ].split("aggregatedOffers")[0][:-2])

In [63]:
res = ads[0]

In [65]:
ad_url = res['fullUrl']
ad_id = res['id']
date = res['creationDate']

address = res['geo']['userInput']
latitude = res['geo']['coordinates']['lat']
longitude = res['geo']['coordinates']['lng']
build_year = res['building']['buildYear']
description = res['description'].replace('\n', ' ')

In [68]:
description

'Помещение свободного назначения.  м. Речной вокзал, 5 мин. на авто, ул. Фестивальная, д. 20, к.2. Площадь 421,0 кв.м., 1-й этаж. Два входа/выхода. Идеально под продовольственный магазин, продажу мебели, досуговый центр, салон красоты, массажный салон, пункта приема и выдачи заказов. Находится на проезжей части дороги с хорошей проходимостью. Стихийная парковка. Коммунальные платежи оплачиваются отдельно.  Прямая аренда.'